In [ ]:
import polars as pl
import polars_ds as pld
import numpy as np

In [ ]:
df = pl.DataFrame({
    "a": ["thank_ad", "thankAd", "ThankAd"] * 10_000
}).with_columns(
    pl.col("a").stats.rand_str().alias("s1"),
    pl.col("a").stats.rand_str().alias("s2"),
)
df.head()

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.levenshtein("abcde")
) # 3.26ms

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.levenshtein(pl.col("s2")) < 3
) # 3.25ms

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.levenshtein_filter(pl.col("s2"), 3)
) # 2.43ms